In [1]:
# Vu Quang Hiep - 24C12007
# Course: Multimedia systems
# STEP 01
# Check whether GPU is on yet
import torch
print(torch.cuda.is_available())

True


In [2]:
# STEP 02
# Load AudioLDM with optimized scheduler
from diffusers import AudioLDMPipeline, DPMSolverMultistepScheduler
import torch

# Select large version of AudioLDM, provide higher quality but require more RAM
MODEL_ID = "cvssp/audioldm-l-full"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# Load pipeline
pipe = AudioLDMPipeline.from_pretrained(MODEL_ID, torch_dtype=dtype).to(device)

# Replace scheduler for better quality and smoother output
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

print(f"Loaded: {MODEL_ID} | scheduler={pipe.scheduler.__class__.__name__} | device={device} | dtype={dtype}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

text_encoder/pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cvssp--audioldm-l-full/snapshots/26aec6fc5bb4f855a11e8357e0d69c8dc9d0ab95/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cvssp--audioldm-l-full/snapshots/26aec6fc5bb4f855a11e8357e0d69c8dc9d0ab95/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
`torch_dtype` is deprecated! Use `dtype` instead!
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cvssp--audioldm-l-full/snapshots/26aec6fc5bb4f855a11e8357e0d69c8dc9d0ab95/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cvssp--audioldm-l-full/snapshots/26aec6fc5bb4f855a11e8357e0d69c8dc9d0ab95/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The AudioLDMPipeline has been deprecated and will not receive bug fixes or f

Loaded: cvssp/audioldm-l-full | scheduler=DPMSolverMultistepScheduler | device=cuda | dtype=torch.float16


In [3]:
# STEP 03
# Generate audio function
import os, time
import numpy as np
import scipy.io.wavfile as wavfile
from IPython.display import Audio, display
import torch

def generate_audio(
    prompt: str,
    seconds: float = 10.0,    # audio length
    steps: int = 300,         # denoising steps number, the more steps the better audio quality, but require more GPU
    guidance: float = 3.2,    # prompt adherence setting, higher is stricter
    seed: int | None = None,
    sample_rate: int = 16000
):
    if not prompt or not prompt.strip():
        raise ValueError("Error: Empty sound description")

    # Random seed for reproducibility
    gen = torch.Generator(device=pipe.device)
    if seed is None:
        seed = int.from_bytes(os.urandom(2), "big")
    gen.manual_seed(seed)

    # Run pipline to generate audio
    out = pipe(
        prompt,
        num_inference_steps=int(steps),
        audio_length_in_s=float(seconds),
        guidance_scale=float(guidance),
        generator=gen,
    )
    audio = out.audios[0]

    # Save WAV file and play the audio immediately
    ts = int(time.time())
    filename = f"/content/audioldm_{ts}_{seed}.wav"
    wavfile.write(filename, rate=sample_rate, data=audio)

    display(Audio(audio, rate=sample_rate, autoplay=True))
    print(f"✅ Saved: {filename} | Seed: {seed} | steps={steps} | guidance={guidance}")
    return filename


In [ ]:
# STEP 04 (Optional)
# Enter sound description to call the generate_audio function to generate output
user_prompt = input("Enter sound description (EN or VI): ")
_ = generate_audio(
    prompt=user_prompt,
    seconds=10.0,
    steps=200,
    guidance=3.5,
    seed=None
)

KeyboardInterrupt: Interrupted by user

In [4]:
# STEP 05
# Install necessary packages for FastAPI app and expose it online using ngrok
!pip -q install fastapi uvicorn nest_asyncio pyngrok

# Set ngrok authtoken
!ngrok config add-authtoken 33dMioLfi4KE7L84JT0gIYUOtbY_6JV8eczSNdNB1rL7cMznn

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
# STEP 06
import os
# Setting google colab API key (required in header when sending request from client)
os.environ["COLAB_API_KEY"] = "multimedia-482jf2l4512"

In [16]:
# STEP 07
# Start API server with ngrok
import os, re, nest_asyncio
from fastapi import FastAPI, Header, HTTPException
from fastapi.responses import FileResponse, JSONResponse
from pydantic import BaseModel, Field
from pyngrok import ngrok
import uvicorn
import asyncio

# API key auth
API_KEY = os.environ.get("COLAB_API_KEY", "multimedia-482jf2l4512")
def check_key(x_api_key: str | None):
    if (x_api_key or "") != API_KEY:
        raise HTTPException(status_code=401, detail="Unauthorized")

# Request schema
class GenRequest(BaseModel):
    prompt: str = Field(..., description="Sound description")
    seconds: float = Field(10.0, ge=1, le=30)
    steps: int = Field(200, ge=20, le=400)
    guidance: float = Field(3.2, ge=0.0, le=10.0)
    seed: int | None = None

app = FastAPI(title="AudioLDM Colab API", version="1.0")
AUDIO_ROOT = "/content"

# Expose generate audio endpoint
@app.post("/generate")
def generate(req: GenRequest, x_api_key: str | None = Header(default=None)):
    check_key(x_api_key)

    filename = generate_audio(
        prompt=req.prompt,
        seconds=req.seconds,
        steps=req.steps,
        guidance=req.guidance,
        seed=req.seed
    )
    short = os.path.basename(filename)
    download_url = f"{PUBLIC_URL}/download/{short}"

    return JSONResponse({
        "filename": short,
        "seconds": req.seconds,
        "steps": req.steps,
        "guidance": req.guidance,
        "seed": req.seed,
        "download_url": download_url
    })

# Expose download audio file endpoint
@app.get("/download/{fileName}")
def download(fileName: str, x_api_key: str | None = Header(default=None)):
    check_key(x_api_key)
    if not re.fullmatch(r"[A-Za-z0-9_\-\.]+", fileName):
        raise HTTPException(status_code=400, detail="Invalid filename")
    full = os.path.join(AUDIO_ROOT, fileName)
    if not os.path.exists(full):
        raise HTTPException(status_code=404, detail="File not found")
    return FileResponse(full, media_type="audio/wav", filename=fileName)

# Create public tunnel to local port 8000
http_tunnel = ngrok.connect(8000)
PUBLIC_URL = http_tunnel.public_url

print("✅  PUBLIC_URL:", PUBLIC_URL)
print("➡️  POST generate:", f"{PUBLIC_URL}/generate")
print("➡️  GET  download:", f"{PUBLIC_URL}/download/&lt;filename&gt;")

# Run uvicorn in the notebook to activate and keep this server active
config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
await server.serve()

✅  PUBLIC_URL: https://tristen-symbolistic-crystal.ngrok-free.dev
➡️  POST generate: https://tristen-symbolistic-crystal.ngrok-free.dev/generate
➡️  GET  download: https://tristen-symbolistic-crystal.ngrok-free.dev/download/&lt;filename&gt;


INFO:     Started server process [1125]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


  0%|          | 0/200 [00:00<?, ?it/s]

✅ Saved: /content/audioldm_1761128083_19915.wav | Seed: 19915 | steps=200 | guidance=3.2
INFO:     113.161.95.160:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     113.161.95.160:0 - "GET /download/audioldm_1761128083_19915.wav HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1125]


In [ ]:
# Clean all the wav files
!rm /content/*.wav